<a href="https://colab.research.google.com/github/johannes-kk/am207/blob/master/exercises/20_Understanding_VAEs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAY 20: Understanding VAEs


### AM207: Advanced Scientific Computing

#### Instructor: Weiwei Pan

#### Due: November 10th, 11:59pm EST

**Names of Group Members**:
- Matthieu Meeus (matthieu_meeus@g.harvard.edu)
- Nari Johnson njohnson@college.harvard.edu
- Maggie Wang maggiewang@college.harvard.edu
- Will Seaton (wseaton@g.harvard.edu)
- Alex Spiride (aspiride@college.harvard.edu)
- Johannes Kolberg (johanneskolberg@g.harvard.edu)

## Learning Goals:

1. Gain intuition for each step in the derivation of the VAE ELBO
2. Gain intuition for each term in the ELBO
3. Gain intuition for when a VAE model might be appropriate for the data


### Load necessary libraries

In [ ]:
### Import basic libraries
import sys
from scipy.special import logsumexp
from autograd import numpy as np
from autograd import scipy as sp
from autograd import grad
from autograd.misc.optimizers import adam, sgd
from autograd.scipy.stats import multivariate_normal
import numpy
import math
import pdb
import matplotlib.pyplot as plt
%matplotlib inline

## Problem: Walking Through the Derivation of the VAE Training Objective

Variational Autoencoders are unique amongst the models we've studied in this course because VAEs imply both a model ***and*** an inference method! The generative model of a VAE is actually very simple, it's a latent variable model very similar to Gaussian mixture models, factor analysis etc.

<img src="https://i.ibb.co/7zG9m5z/Untitled-11.png">

But to say that your are working with a VAE, you are also automatically committed to performing a type of inference: a modified form of EM. 

**Exercise 1:** The goal of VAE inference is to learn 

$$
\theta_{\mathrm{MLE}} = \mathrm{argmax}_\theta \log \prod_{n=1}^N p_\theta(y_n)
$$

The entire lecture involves coming up with a way to avoid directly optimizing the observed data log-likelihood. Why is that, i.e. why don't we directly maximize the observed data log-likelihood?

**Answer:**


If we tried to directly maximize the observed data log-likelihood, we would need to work with a gradient involving an expectation. That means it must be estimated instead of calculated directly. But when we look at the form of our gradient, we see that estimating using a method like Monte Carlo will leave us with very high variance. This will prevent our sampler from converging or will cause it to converge very slowly or even not at all. The high variance will be produced by the below formula of the gradient.

$$\Sigma_{n=1}^N \frac{\Delta_{\theta} \mathbb{E}_{p(Z_n)} \left[ p_W(y_n | Z_n) \right]}{\mathbb{E}_{p(Z_n)} \left[ p_W(y_n | Z_n) \right]}$$ 

**Exercise 2:** Can we optimize $\prod_{n=1}^N p_\theta(y_n)$ directly if $p(z)$ does not depend on $\theta$?

**Answer:**

We believe the answer is yes. When we take the gradient as specified above in exercise 1, $p(z)$ will not depend on $\theta$ anymore. Hence, we could move the gradient behing the expectation and thus make the computation of the gradient and the thus the optimization possible! 

Again, the reason we don't do so in VAE (where it $p(z)$ is indeed a normal distribution that does not depend on $\theta$, is because of the high variance we get in the Monte Carlo estimates of the gradient. it has been proven that even with variance reduction methods, this does not work well. With this, the need for another approach clearly rises! 

**Exercise 3:** Where is Jensen's inequality used and why?

**Answer:**

Jensen's Inequality is used when we're maximizing the ELBO to approximate the MLE of the weights of our Deep Generative Model. It allows us to pass our log into the expectation we are taking of our posterior over our importance distribution. Passing in the log turns an equivalence into a lower bound inequality, so we are now saying the ELBO can be no less than this, instead of saying the optimization is exactly this.  $$=\Sigma^N_{n=1} log \mathbb{E}_{q(Z_n)} \left[ \frac{p_W(y_n | Z_n)p(Z_n)}{q(Z_n} \right] $$ becomes via Jensen's Inequality $$ \ge \Sigma^N_{n=1} \mathbb{E}_{q(Z_n)} \left[ log \frac{p_W(y_n | Z_n)p(Z_n)}{q(Z_n} \right] $$ 


**Exercise 4:** If we're interested in maximizing the observed data log-likelihood, why is it valid to maximize a lower bound?

**Answer:**

It's valid to maximize a lower bound because it provides a guarantee that our log-likelihood will be at least as high as the optimal lower bound that we find. Since maximizing the actual log likelihood is hard, we use the lower bound becuase it's easier. If we arrive at some maximum lower bound, then that's the tightest bound we can get on our log-likelihood and ensures that the minimum of the range of possible log-likelihoods is as large as possible.

The point is that computing the lower bound is feasible and while it is not equivalant to finding the maximum log-likelihood, it is a fair thing to do. 

**Exercise 5:** Why do we need to optimize over $q$? Why can't we simply fix a single $q$, like a standard normal distribution?

**Answer:**

Normally in the E-step, we would compute $q$ directly from this equation:

$$q = p_{W^*}(Z_n | y_n)$$

But for a neural network, ie when  $y_n = nn.forward(Z_n, W^*) + \epsilon$, then the posterior is intractable in closed form.

Hence, we need to come up with an approximation of this true posterior using some kind of variational inference. This makes that we need to optimize for the variables defining our variational distribution, thus $\mu$ and $\Sigma$ in order to match our posterior as closely as possible. 


**Exercise 6:** What framework of optimization is used to optimize the VAE EBLO? How does this framework relate to other inference frameworks you've seen in this course (**Hint:** describe exactly how VAE inference is related to EM)?

**Answer:**

 We use gradient descent as our framework of optimization. Specifically, we simultaneously iterate over the weights (W) and parameters (V) that define the function $g_{V*}(y_n)$. We can use an automatic differentiation package like `autograd` over our gradient, which is define as: $$ \Delta_{W,V}ELBO(W, q_V) = \Sigma_{n=1}^N \mathbb{E}_{\epsilon \sim N(0,I)} \left[ \Delta_{W,V}  \frac{P_W(\epsilon \Sigma_V*{1/2}(y_n) + \mu_V(y_n), y_n}{q_V(\epsilon \Sigma_V*{1/2}(y_n) + \mu_V(y_n)} \right] $$ 

 This is slightly different from EM as we are able to combine both the E and the M step, as we can compute the gradient with respect to both variables W and V at the same time. In traditional EM, we need gradient descent in the M step and can update an optimal q using a closed form of the posterior. For VAE ELBO, this closed form does not exist, thus requiring an additional optimization problem for $V$ (through the Amortized inference trick), which makes it possible to combine the gradient descents in both E and M steps into one framework. 

**Exercise 7:** At what point of the optimization is variational inference needed? Why is variational inference needed? 

**Answer:**

We need variational inference when dealing with a neural network form with random error $y_n = nn.forward(Z_n, W^*) + \epsilon$. This is because $p_{W^*}(Z_n | y_n)$ is intractable in closed form when calculating the E-Step of expectation maximization. By using variational inference, choosing a mean-field variational family $q(Z_n) = N(Z_n; \mu_n, \Sigma_n)$ with $\Sigma_n$ being a diagonal matrix, we can now approximate the posterior using a form that we're better able to work with. Specifically, we are doing: $$ \underset{\mu_n, \Sigma_n}{argmax} \mathbb{E}_{q_{\mu_n, \Sigma_n}} \left[ log \frac{p_{W^*}(y_n | Z_n)p(Z_n)}{q_{\mu_n, \Sigma_n}(Z_n)} \right] $$


**Exercise 8:** Describe what is amortized inference and why we need it for VAE training (what problem does it solve?). Under what assumptions is amortized inference valid? Give an example of $f_\theta$ under which amortized inference is valid and give an example of $f_\theta$ under which amortized inference is not valid.

**Answer:**

Normally, we would need to compute the VI approximation for the true posterior for each data point that we have. This makes it computationally very expensive and in most cases not feasible. 

We then reason that the posterior of 'similar' data points should have 'similar' posteriors and that we could come up with a reasonable function $g_{V}$ that spits out the variational parameters $\mu$ and $\Sigma$ for a given data point. This is exactly what we call Amortized Inference and this will enable us to combine the E and M steps in a combined gradient descent algorithm. 

It seems that this is not valid when the posteriors are very different for very similar data points, **but not sure how you can evaluate this**? 

**Exercise 9:** Where is the reparametrization trick used in VAE training and why is it used?

**Exercise 10:** Compare VAE ELBO to the ELBO for variational inference, what are the similarities and differences between these two ELBOs?

## Problem 2: Interpreting the VAE ELBO

In this problem, you'll build some intuition for what the VAE training objective is trying to accomplish.

**Exercise 11:** In the lecture, we discussed how the VAE ELBO can be written in different ways, each of which facilitates different computational aspects of training. 

Interpret each term in ELBO presentation II, describe intuitively what is each term trying to accomplish. Are the goals of the two terms complimentary or in tension? 

Interpret each term in ELBO presentation III, describe intuitively what is each term trying to accomplish. Are the goals of the two terms complimentary or in tension? 

Given your interpretation of the VAE ELBO, can you forsee any potential issues during training (any undesirable optima of the ELBO)?

**Exercise 12:** VAEs are often compared to autoencoders, a type of neural network architecture with two parts: the first (encoder) encodes observations as (usually) lower-dimensional latent representations; the second (decoder) decodes latent representation into observations. 

The encoder and decoder are parametrized as neural networks, $f_\theta$ and $g_\phi$ respectively. Both networks are trained jointly to minimize the **reconstruction error**:

$$
\theta^*, \phi^* = \mathrm{argmin}_{\theta, \phi} \frac{1}{N} \sum_{n=1}^N (y_n - g_\phi \circ f_\theta(y_n))^2
$$

<img src="https://i.ibb.co/VVbSV3T/autoencoder.png">

In what sense are VAEs similar to autoencoders?

Despite their similarity, we claim that VAEs and autoencoders are fundamentally different in their goals. Compare the two training objects and justify our claim. 

For practitioners, why is it important to understand how VAEs are different from autoencoders? Think about for which tasks autoencoders are appropriate and for which tasks VAEs might be more appropriate.

**Answer:**

VAEs are similar to autoencoders in that the both involve taking the data, encoding it down into a latent space, and then decoding it back to the original input. Their architectures are roughly the same.

However, these are fundamentally different. In autoencoders, the goal is to compress the data as accurately as possible into the (uninterpretable) latent space. You have no control over the compression into the latent space - your only objective is to minimize reconstruction error (and thus compress the data with as little reconstruction loss as possible).

With VAEs, we aim to learn the parameters of the latent distribution that describe the data. The ultimate goal is to have a distribution that can model our data. The VAE is a deep generative model - once we learn the parameters, we can sample from the distribution to generate alternative data along our chosen parameters.

In situations in which we want precise control over our latent distribution, such as wanting to be able to sample from the distribution, we should use VAEs because we can specify the parameters. In situations in which we don't care what the latent distribution represents or we don't want to sample, such as only caring about compressing the data, then AEs suffice.

With the faces example, we could use AEs to perform dimensionality reduction on the dataset, then feed the uninterpretable latent space into a neural network to perform classification on the faces. This way, our problem is in a reduced dimensional space, but we know that the latent space minimizes reconstruction error: it encodes as much of the original data as possible.

We'd use VAEs for the faces example to generate new faces from the distribution. I might wonder what I'd look like with glasses, or if I were older. I can specifically encode these variables in the latent distribution (age and glasses) among others, and then generate an image with my latent variables, only the age parameter is increased. I can generate multiple samples to find different possibilities for how I might age.